In [386]:
import os
import warnings
import requests
import string 
import numpy as np
import nltk
import time
from numpy import linalg as LA
import regex as re
import pandas as pd
from lda import LDA
from datetime import datetime
from bs4 import BeautifulSoup
from alive_progress import alive_bar
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
from selenium.webdriver.chrome.options import Options
from alive_progress import alive_bar

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)


In [387]:
ROOT_DIR = os.path.dirname(os.path.abspath("__file__"))
PARENT_DIR = os.path.dirname(ROOT_DIR)
GUARDIAN_DIR = os.path.join(ROOT_DIR, "data", "Guardian.csv")
REUTERS_DIR = os.path.join(ROOT_DIR, "data", "Reuters.csv")
CNN_DIR = os.path.join(PARENT_DIR, "data", "CNN.csv")
GLOVE_DIR = os.path.join(ROOT_DIR, "glove_data", "results", "vectors.txt")

# CNN

In [388]:
s = Service(ChromeDriverManager(chrome_type=ChromeType.BRAVE, path=ROOT_DIR).install())
o = webdriver.ChromeOptions()
o.add_argument('headless')
driver = webdriver.Chrome(service=s,options=o)
ignored_exceptions = (
    NoSuchElementException,
    StaleElementReferenceException,
)
wait2 = WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions)

In [389]:
class CNN: 
    def __init__(self) -> None:
        self.source = "CNN"

    def fromScratch(self):
            if not os.path.exists(CNN_DIR):
                self.old_data = pd.DataFrame(columns=["Date", "URL", "Title", "Text"])
                self.from_scratch = True
            else:
                self.old_data = pd.read_csv(CNN_DIR)
                self.from_scratch = False

    def concatData(self):
        result = pd.concat([self.old_data, self.new_data])
        result = result.drop_duplicates(subset=["Text"])
        result = result.set_index("Date")
        result = result.sort_index(ascending=False)
        return result

    def URLFetcher(self):
            self.urls = []
            self.dates = []

            if self.from_scratch == False:
                last_url = self.old_data.iloc[0,1]
            elif self.from_scratch == True:
                last_url = "https://www.cnn.com/2021/05/11/politics/romania-nato-exercises-russia/index.html"

            with alive_bar(title=f"→ {self.source}: Fetching URLs in pages", bar=None, spinner="dots", force_tty=True) as bar:
                for page in range(0,10): # 95
                    url = "https://edition.cnn.com/search?q=ukraine+russia&from="+str(page*50)+"&size=50&page=1&sort=newest&types=article&section="
                    title_tag="//div//a[@class='container__link __link']"
                    exc_list=['/tennis/','/live-news/','/opinions/','/tech/','/sport/','/us/','/football/','/china/','/style/','/business-food/','/americas/','/travel/','/business/']
                    inc_list=['/2022/','/2021/']
                    driver.get(url)
                    try:
                        titles = wait2.until(EC.presence_of_all_elements_located((By.XPATH, title_tag))) 
                        for title in titles:
                            url = title.get_attribute("href")
                            if not any(s in url for s in exc_list) and any(s in url for s in inc_list):
                                self.urls.append(url)
                            if last_url == url:
                                break
                        if last_url == url:
                            break
                    except Exception as e:
                        print(f"Error in page {page}: {e}")
                        pass
                    bar()
                print(f"latest page: {page}")
                driver.quit()
            self.unique_urls = list(dict.fromkeys(self.urls))
            print(f"-> {len(self.unique_urls)} URLs fetched successfully!")

    def articleParser(self):
        bodies = []
        titles = []
        dates = []
        urls = []
        rep = {"This story has been updated with additional information.": ""}

        def replaceAll(text, dic):
            for i, j in dic.items():
                text = text.replace(i, j)
            return text

        with alive_bar(len(self.unique_urls), title=f"→ {self.source}: Scraper", spinner="dots_waves", bar="smooth", force_tty=True) as bar:
            for url in self.unique_urls:
                try:
                    title_tags = ["pg-headline"]
                    text_tags = ["zn-body__paragraph"]
                    opener_tag = ["zn-body__paragraph speakable"]
                    html_text = requests.get(url).text
                    soup = BeautifulSoup(html_text, "lxml")
                    title = soup.find("h1", class_=title_tags).text
                    opener = soup.find("p", class_=opener_tag).text
                    paragraphs = soup.find_all("div", class_=text_tags)
                    body = opener + ""
                    for _ in paragraphs:
                        body += " " + _.text
                    body = replaceAll(body, rep)
                    bodies.append(re.sub(r"^[^\)]*\)", "", " ".join(body.split()))) # Local tag
                    titles.append(title)
                    urls.append(url)
                    dates.append(url[20:][:10].replace("/","-"))
                    bar()
                except Exception as e:
                    print(f"URL couldn't be parsed: {url} because {e}")
                    pass
        data = pd.DataFrame({"URL": urls, "Date": dates, "Title": titles, "Text": bodies})
        self.new_data = data

    def scraper(self):
        self.fromScratch()
        self.URLFetcher()
        self.articleParser()
        data = self.concatData()
        lenAfter = len(data) - len(self.old_data)
        if lenAfter == 0:
            print(f"→ No new articles found. Total articles: {len(data)}")
        else:
            print(f"→ {lenAfter} new articles saved to {self.source}.csv! Total articles: {len(data)}")
        data.to_csv(CNN_DIR, index=True)

        return data

In [390]:
cnn= CNN()
cnn.scraper()

https://www.cnn.com/2022/08/27/europe/darya-dugina-russia-disinformation-women-intl-cmd/index.html
on 0: break 1                                                                                                           
on 0: break 2                                                                                                           
on 0: latest page: 0                                                                                                    
→ CNN: Fetching URLs in pages (!) 0 in 2.5s (0.00/s) 
-> 3 URLs fetched successfully!
→ CNN: Scraper |████████████████████████████████████████| 3/3 [100%] in 1.3s (2.25/s)                                   
→ No new articles found. Total articles: 92


,URL,Title,Text
Date,,,
2022-08-27,https://www.cnn.com/2022/08/27/europe/darya-du...,Darya Dugina's death provides a glimpse into R...,When a car bomb exploded on the outskirts of M...
2022-08-27,https://www.cnn.com/2022/08/27/politics/russia...,How Ukraine is using resistance warfare develo...,As the war in Ukraine has passed the six-month...
2022-08-27,https://www.cnn.com/2022/08/27/europe/russia-b...,Russia blocks final draft of nuclear disarmame...,A month-long meeting on nuclear disarmament en...
2022-08-27,https://www.cnn.com/2022/08/27/europe/russia-u...,Donetsk People's Republic says it is ready to ...,The Donetsk People's Republic (DPR) says it is...
2022-08-26,https://www.cnn.com/2022/08/26/europe/ukraine-...,Ukraine's Zaporizhzhia nuclear power plant rec...,"Ukraine's Zaporizhzhia nuclear power plant, wh..."
...,...,...,...
2022-08-01,https://www.cnn.com/2022/08/01/politics/blinke...,"Blinken accuses Russia of 'reckless, dangerous...",Secretary of State Antony Blinken on Monday ca...
2022-08-01,https://www.cnn.com/2022/08/01/politics/steve-...,Attorney for convicted Russian arms dealer Vik...,An attorney for notorious Russian arms dealer ...
2022-07-31,https://www.cnn.com/2022/07/31/africa/sudan-pr...,Sudan's military leaders launch 'manhunt' for ...,"Sudan's military authorities have launched a ""..."
